In [1]:
import pickle
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
with open('data\city_db.pkl', 'rb') as file:
    cities = pickle.load(file)

with open('data\met_data.pkl', 'rb') as file:
    met_data = pickle.load(file)

In [51]:
# Incomplete ideas for feature engineering modularization

# def calculate_feature(group, variable):
#     match variable:
#         case 'max':
#             return group.max()
#         case 'min':
#             return group.min()
#         case 'std':
#             return group.std()
#         case 'avg':
#             return group.mean()
#         case 'amplitude':
#             return group.max() - group.min()

# def group_variables(data, variable, features=None):
#     if not features:
#         features = ['max', 'min', 'std', 'avg', 'amplitude']
    
#     group = data.groupby([pd.Grouper(freq='D')])
#     for var in data.columns:
#         calculate_feature(group, variable):

In [53]:
daily_city_stats = {}
for city_id, data in met_data.items():
    
    # Make preprocessings like this before data consumption
    data = data.loc[:, ['temp','prcp']]
    data.fillna(0, inplace=True)
    # -----

    data['prcp_shift'] = data['prcp'].shift()
    data['rain_start_count'] = 0
    data.loc[(data['prcp'] > 0) & (data['prcp_shift']==0), 'rain_start_count'] = 1

    daily_group = data.groupby([pd.Grouper(freq='D')])
    daily_stats = {
        'temp':{
            'max':daily_group['temp'].max(),
            'min':daily_group['temp'].min(),
            'avg':daily_group['temp'].mean(),
            'std':daily_group['temp'].std(),
            'amplitude':daily_group['temp'].max() - daily_group['temp'].min()
        },
        'prcp':{
            'cumsum':daily_group['prcp'].sum(),
            'n_rain_starts':daily_group['rain_start_count'].sum()
        }
    }

    daily_city_stats[city_id] = daily_stats

with open('data\daily_city_stats.pkl', 'wb') as file:
    pickle.dump(daily_city_stats, file)

In [76]:
mean_rain_starts = []
mean_temp = []
mean_temp_amplitude = []
mean_temp_std = []

for city_id, daily_stats in daily_city_stats.items():
    mean_rain_starts.append(daily_stats['prcp']['n_rain_starts'].mean())
    mean_temp_amplitude.append(daily_stats['temp']['amplitude'].mean())
    mean_temp.append(daily_stats['temp']['avg'].mean())
    mean_temp_std.append(daily_stats['temp']['std'].mean())

df = pd.DataFrame()
df.index = daily_city_stats.keys()
df['avg_n_rain_starts'] = mean_rain_starts
df['mean_temp_amplitude'] = mean_temp_amplitude
df['mean_temp'] = mean_temp
df['mean_temp_std'] = mean_temp_std

df = df.merge(cities[['Cidade', 'Estado']], how='left', left_index=True, right_index=True)

In [79]:
df.sort_values('mean_temp_std', ascending=False)

,avg_n_rain_starts,mean_temp_amplitude,mean_temp,mean_temp_std,Cidade,Estado
8,0.957447,12.873404,23.704376,4.403597,Goiânia,Goiás
11,0.941973,10.963443,25.166917,3.746102,Campo Grande,Mato Grosso do Sul
6,1.001934,11.075338,21.851966,3.731139,Brasília,Distrito Federal
10,0.980658,10.519826,27.194943,3.560593,Cuiabá,Mato Grosso
12,0.820116,10.323211,21.777212,3.488637,Belo Horizonte,Minas Gerais
26,1.164410,10.230271,27.117775,3.442267,Palmas,Tocantins
20,0.878143,10.656093,19.575814,3.434927,Porto Alegre,Rio Grande do Sul
17,1.341393,9.671760,28.151604,3.287451,Teresina,Piauí
24,1.113153,9.465861,19.119318,3.212519,São Paulo,São Paulo
21,1.369439,8.904836,26.210848,3.029238,Porto Velho,Rondônia
